In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.8 MB/s eta 0:00:0

In [ ]:
import torch
from torch import nn
from transformers import (
    ViltProcessor,
    ViltModel,
    ViltConfig,
    AutoTokenizer,
    RobertaModel,
)
from PIL import Image
import requests
from tqdm.notebook import tqdm
from datasets import load_dataset
from PIL import Image
import io
import json
from google.colab import drive
import sys

config = ViltConfig.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [ ]:
drive.mount("/content/gdrive")

# tools_path = '/content/gdrive/MyDrive/project/VQA-master'
tools_path = "./VQA"

sys.path.append(tools_path + "/PythonEvaluationTools")
sys.path.append(tools_path + "/PythonHelperTools")

from vqaEvaluation import vqaEval
from vqaTools import vqa

Mounted at /content/gdrive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# prepare image and text
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = "How many cats are there?"

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltModel.from_pretrained(
    "dandelin/vilt-b32-finetuned-vqa",
    num_labels=len(config.id2label),
    id2label=config.id2label,
    label2id=config.label2id,
).to(device)

inputs = processor(image, text, return_tensors="pt")
inputs.to(device)
outputs = model(**inputs)

Some weights of the model checkpoint at dandelin/vilt-b32-finetuned-vqa were not used when initializing ViltModel: ['classifier.1.weight', 'classifier.3.weight', 'classifier.1.bias', 'classifier.0.bias', 'classifier.3.bias', 'classifier.0.weight']
- This IS expected if you are initializing ViltModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViltModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(model)

ViltModel(
  (embeddings): ViltEmbeddings(
    (text_embeddings): TextEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(40, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (patch_embeddings): ViltPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
    )
    (token_type_embeddings): Embedding(2, 768)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViltEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViltLayer(
        (attention): ViltAttention(
          (attention): ViltSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=Fa

In [ ]:
dataset = load_dataset("Multimodal-Fatima/VQAv2_validation")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/214354 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Multimodal-Fatima___parquet/Multimodal-Fatima--VQAv2_validation-d29f379b20185d94/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
example = dataset["validation"][0]
print(example.keys())

dict_keys(['question_type', 'multiple_choice_answer', 'answers', 'answers_original', 'id_image', 'answer_type', 'question_id', 'question', 'image', 'id', 'clip_tags_ViT_L_14', 'blip_caption', 'LLM_Description_gpt3_downstream_tasks_visual_genome_ViT_L_14', 'DETA_detections_deta_swin_large_o365_coco_classes', 'DETA_detections_deta_swin_large_o365_clip_ViT_L_14', 'DETA_detections_deta_swin_large_o365_clip_ViT_L_14_blip_caption', 'Attributes_ViT_L_14_descriptors_text_davinci_003_full', 'clip_tags_ViT_L_14_wo_openai', 'clip_tags_ViT_L_14_with_openai', 'clip_tags_LAION_ViT_H_14_2B_wo_openai', 'clip_tags_LAION_ViT_H_14_2B_with_openai', 'clip_tags_LAION_ViT_bigG_14_2B_wo_openai', 'clip_tags_LAION_ViT_bigG_14_2B_with_openai', 'Attributes_LAION_ViT_H_14_2B_descriptors_text_davinci_003_full', 'Attributes_LAION_ViT_bigG_14_2B_descriptors_text_davinci_003_full', 'DETA_detections_deta_swin_large_o365_coco_classes_caption_module_random'])


# Modified datasets to use smaller train set

In [ ]:
new_source_dataset_reduced = torch.utils.data.random_split(
    dataset["validation"], [50000, len(dataset["validation"]) - 50000]
)[0]
new_dataset_train, new_dataset_test = torch.utils.data.random_split(
    new_source_dataset_reduced, [45000, 5000]
)
new_dataset = {"train": new_dataset_train, "test": new_dataset_test}

In [ ]:
print(len(new_dataset["train"]))
print(len(new_dataset["test"]))

45000
5000


In [ ]:
def get_score(count: int) -> float:
    return min(1.0, count / 3)

In [ ]:
class VQADataset(torch.utils.data.Dataset):
    """VQA (v2) dataset."""

    def __init__(self, dataset, processor, split):
        self.dataset = dataset[split]
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # get image + text
        # annotation = self.annotations[idx]
        questions = self.dataset[idx]["question"]
        # image = Image.open(io.BytesIO(self.dataset[idx]['image']))
        image = self.dataset[idx]["image"].convert("RGB")
        # print(image)
        text = questions

        answers = self.dataset[idx]["answers"]
        # print(answers)
        answer_count = {}
        for answer in answers:
            answer_count[answer] = answer_count.get(answer, 0) + 1
        labels = []
        scores = []
        for answer in answer_count:
            if answer not in list(config.label2id.keys()):
                continue
            labels.append(config.label2id[answer])
            score = get_score(answer_count[answer])
            scores.append(score)

        encoding = self.processor(
            image, text, padding="max_length", truncation=True, return_tensors="pt"
        )
        # remove batch dimension
        for k, v in encoding.items():
            encoding[k] = v.squeeze()
        # add labels

        # labels = self.dataset[idx]['labels']
        # scores = self.dataset[idx]['scores']
        # based on: https://github.com/dandelin/ViLT/blob/762fd3975c180db6fc88f577cf39549983fa373a/vilt/modules/objectives.py#L301
        targets = torch.zeros(len(config.id2label))
        for label, score in zip(labels, scores):
            targets[label] = score
        encoding["labels"] = targets
        encoding["caption"] = self.dataset[idx]["blip_caption"]
        encoding["question_id"] = self.dataset[idx]["question_id"]

        return encoding

In [ ]:
final_dataset = VQADataset(new_dataset, processor, "train")
final_test_dataset = VQADataset(new_dataset, processor, "test")

In [ ]:
print(new_dataset["train"][0]["image"])

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x360 at 0x7F309A82D030>


In [ ]:
final_dataset[0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'pixel_values', 'pixel_mask', 'labels', 'caption', 'question_id'])

In [ ]:
from torch.utils.data import DataLoader


def collate_fn(batch):
    input_ids = [item["input_ids"] for item in batch]
    pixel_values = [item["pixel_values"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]
    token_type_ids = [item["token_type_ids"] for item in batch]
    labels = [item["labels"] for item in batch]
    captions = [item["caption"] for item in batch]
    question_id = [item["question_id"] for item in batch]

    # create padded pixel values and corresponding pixel mask
    encoding = processor.feature_extractor.pad_and_create_pixel_mask(
        pixel_values, return_tensors="pt"
    )

    # create new batch
    batch = {}
    batch["input_ids"] = torch.stack(input_ids)
    batch["attention_mask"] = torch.stack(attention_mask)
    batch["token_type_ids"] = torch.stack(token_type_ids)
    batch["pixel_values"] = encoding["pixel_values"]
    batch["pixel_mask"] = encoding["pixel_mask"]
    batch["labels"] = torch.stack(labels)
    batch["caption"] = captions
    batch["question_id"] = question_id

    return batch


train_dataloader = DataLoader(
    final_dataset, collate_fn=collate_fn, batch_size=8, shuffle=True
)
test_dataloader = DataLoader(
    final_test_dataset, collate_fn=collate_fn, batch_size=8, shuffle=False
)

In [ ]:
for i, data in enumerate(train_dataloader):
    print(data.keys())
    break

dict_keys(['input_ids', 'attention_mask', 'token_type_ids', 'pixel_values', 'pixel_mask', 'labels', 'caption', 'question_id'])


/usr/local/lib/python3.10/dist-packages/transformers/models/vilt/processing_vilt.py:142: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/vilt/image_processing_vilt.py:357: FutureWarning: This method is deprecated and will be removed in v4.26.0. Please use pad instead.
  warnings.warn(


In [ ]:
classifier = nn.Sequential(
    nn.Linear(in_features=1536, out_features=3072, bias=True),
    nn.LayerNorm((3072,), eps=1e-05, elementwise_affine=True),
    nn.GELU(),
    nn.Linear(in_features=3072, out_features=3129, bias=True),
)

classifier.to(device)

Sequential(
  (0): Linear(in_features=1536, out_features=3072, bias=True)
  (1): LayerNorm((3072,), eps=1e-05, elementwise_affine=True)
  (2): GELU(approximate='none')
  (3): Linear(in_features=3072, out_features=3129, bias=True)
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
caption_model = RobertaModel.from_pretrained("roberta-base")
caption_model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

In [ ]:
def vilt_to_vqa_annotation(vilt_data):
    dummy_annotate = """{"info": {"description": "This is Balanced Binary Abstract Scenes VQA dataset.", 
      "url": "http://visualqa.org", 
      "version": "1.0", 
      "year": "2017", 
      "contributor": "VQA Team", 
      "date_created": "2017-03-09 14:27:27"}, 
      "license": {"url": "http://creativecommons.org/licenses/by/4.0/", 
      "name": "Creative Commons Attribution 4.0 International License"}, 
      "data_subtype": "val2017"}"""

    annotate_full = json.loads(dummy_annotate)

    dummy_questions = """
    {"info": {"description": "This is v1.0 of the VQA dataset.", 
    "url": "http://visualqa.org", 
    "version": "1.0", 
    "year": 2015, "contributor": 
    "VQA Team", "date_created": "2015-10-02 19:50:36"}, 
    "task_type": "Open-Ended", 
    "data_type": "abstract_v002", 
    "license": {"url": "http://creativecommons.org/licenses/by/4.0/", 
      "name": "Creative Commons Attribution 4.0 International License"}, 
    "data_subtype": "val2015"}
    """

    question_full = json.loads(dummy_questions)

    annotate_list = []

    question_list = []

    for datapoint in vilt_data:
        """
        print('========')
        for key in datapoint.keys():
          print(key)
        print('========')
        """

        # VILT converted VQA keys, so we need to convert them back
        datapoint["image_id"] = datapoint["id_image"]
        datapoint["answers"] = datapoint["answers_original"]
        # Throw away unneeded stuff
        datapoint["image"] = None
        datapoint["LLM_Description_gpt3_downstream_tasks_visual_genome_ViT_L_14"] = None
        datapoint["DETA_detections_deta_swin_large_o365_coco_classes"] = None
        datapoint["DETA_detections_deta_swin_large_o365_clip_ViT_L_14"] = None
        datapoint[
            "DETA_detections_deta_swin_large_o365_clip_ViT_L_14_blip_caption"
        ] = None

        question_item = {
            "question_id": datapoint["question_id"],
            "image_id": datapoint["image_id"],
            "question": datapoint["question"],
        }

        question_list.append(question_item)
        annotate_list.append(datapoint)

    annotate_full["annotations"] = annotate_list
    question_full["questions"] = question_list

    return {"annotations": annotate_full, "questions": question_full}

In [ ]:
torch.cuda.empty_cache()

In [ ]:
optimizer = torch.optim.AdamW(classifier.parameters(), lr=5e-5)
classifier.to(device)

classifier.train()
for epoch in range(20):  # loop over the dataset multiple times
    print(f"Epoch: {epoch}")
    for batch in tqdm(train_dataloader):
        caption = batch.pop("caption")
        # print(caption)
        labels = batch.pop("labels")
        labels = labels.to(device)
        question_id = batch.pop("question_id")
        # print(batch)

        # get the inputs;
        batch = {k: v.to(device) for k, v in batch.items()}
        # caption = batch[]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(**batch)

        input_captions = tokenizer(caption, return_tensors="pt", padding=True)
        input_captions = input_captions.to(device)
        caption_output = caption_model(**input_captions)
        # cap =
        new_outputs = torch.cat(
            [outputs.pooler_output, caption_output.pooler_output], dim=1
        )
        # new_outputs.to(device)
        logits = classifier(new_outputs)
        # logits.to(device)
        # labels.to(device)
        # print(labels)
        loss = (
            nn.functional.binary_cross_entropy_with_logits(logits, labels)
            * labels.shape[1]
        )
        # loss = outputs.loss
        print("Loss:", loss.item())
        loss.backward()
        optimizer.step()
    torch.save(
        classifier.state_dict(),
        "/content/gdrive/MyDrive/CSCI 566 project/models/classifier_model_epoch{}.pth".format(
            epoch
        ),
    )
    classifier.eval()
    results = []
    answers = []
    for batch in tqdm(test_dataloader):
        caption = batch.pop("caption")
        # print(caption)
        labels = batch.pop("labels")
        labels = labels.to(device)
        question_id = batch.pop("question_id")

        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

        input_captions = tokenizer(caption, return_tensors="pt", padding=True)
        input_captions = input_captions.to(device)
        caption_output = caption_model(**input_captions)
        # cap =
        new_outputs = torch.cat(
            [outputs.pooler_output, caption_output.pooler_output], dim=1
        )
        # new_outputs.to(device)
        logits = classifier(new_outputs)
        for i in range(len(logits)):
            idx = logits[i].argmax(-1).item()
            # print("Predicted answer:", model.config.id2label[idx])
            answer = model.config.id2label[idx]
            answers.append(answer)

            results.append({"question_id": question_id[i], "answer": answer})

    convert_data = vilt_to_vqa_annotation(new_dataset["test"])

    convert_annotate, convert_questions = (
        convert_data["annotations"],
        convert_data["questions"],
    )

    with open("./annotate.json", "w") as annotate_file:
        annotate_file.write(json.dumps(convert_annotate))

    with open("./questions.json", "w") as annotate_file:
        annotate_file.write(json.dumps(convert_questions))

    with open("./results.json", "w") as res_file:
        res_file.write(json.dumps(results))

Epoch: 0


  0%|          | 0/5625 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vilt/processing_vilt.py:142: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/vilt/image_processing_vilt.py:357: FutureWarning: This method is deprecated and will be removed in v4.26.0. Please use pad instead.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Loss: 3.3699629306793213
Loss: 2.084883213043213
Loss: 4.15392541885376
Loss: 0.41783463954925537
Loss: 0.9744450449943542
Loss: 1.7792128324508667
Loss: 2.8758788108825684
Loss: 3.0214710235595703
Loss: 3.9924533367156982
Loss: 1.6640629768371582
Loss: 0.6964738368988037
Loss: 6.717115879058838
Loss: 2.2204906940460205
Loss: 4.683872699737549
Loss: 3.520021677017212
Loss: 2.8704721927642822
Loss: 2.7370681762695312
Loss: 2.0691120624542236
Loss: 3.7313666343688965
Loss: 2.3857250213623047
Loss: 2.0940864086151123
Loss: 4.077233791351318
Loss: 1.620634913444519
Loss: 1.4809086322784424
Loss: 1.4268351793289185
Loss: 2.789017915725708
Loss: 2.729828357696533
Loss: 4.434778690338135
Loss: 3.9545514583587646
Loss: 2.059457302093506
Loss: 2.228743553161621
Loss: 3.3711097240448
Loss: 1.994597315788269
Loss: 1.2105497121810913
Loss: 2.628988265991211
Loss: 3.446512222290039
Loss: 2.887235403060913
Loss: 2.023163318634033
Loss: 2.09639430046

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch: 1


  0%|          | 0/5625 [00:00<?, ?it/s]

Loss: 1.3101356029510498
Loss: 2.2064878940582275
Loss: 2.333846092224121
Loss: 0.5318339467048645
Loss: 1.599511981010437
Loss: 0.7978501915931702
Loss: 3.132147789001465
Loss: 1.9176372289657593
Loss: 3.3210015296936035
Loss: 2.5478358268737793
Loss: 3.2330760955810547
Loss: 4.035234451293945
Loss: 1.3052741289138794
Loss: 0.9388899803161621
Loss: 1.4136149883270264
Loss: 2.7568578720092773
Loss: 2.6303536891937256
Loss: 1.7895419597625732
Loss: 2.486201286315918
Loss: 1.4518215656280518
Loss: 2.3025968074798584
Loss: 4.461450576782227
Loss: 2.0506834983825684
Loss: 3.776693820953369
Loss: 2.835289478302002
Loss: 1.5963985919952393
Loss: 4.269031524658203
Loss: 1.399706482887268
Loss: 2.0764009952545166
Loss: 4.936296463012695
Loss: 3.7614691257476807
Loss: 1.9537768363952637
Loss: 1.9084478616714478
Loss: 0.8783009648323059
Loss: 1.842902660369873
Loss: 2.039971351623535
Loss: 1.4998524188995361
Loss: 2.015521287918091
Loss: 0.660785973072052
Loss: 1.2807824611663818
Loss: 2.6004359

In [ ]:
len(test_dataloader)

625

In [ ]:
# Try evaluation
optimizer = torch.optim.AdamW(classifier.parameters(), lr=5e-5)
classifier.to(device)

classifier.load_state_dict(
    torch.load(
        "/content/gdrive/MyDrive/CSCI 566 project/models/classifier_model_epoch3.pth"
    )
)
classifier.eval()

results = []
answers = []

for batch in tqdm(test_dataloader):
    caption = batch.pop("caption")
    labels = batch.pop("labels")
    labels = labels.to(device)
    question_id = batch.pop("question_id")

    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)

    input_captions = tokenizer(caption, return_tensors="pt", padding=True)
    input_captions = input_captions.to(device)
    caption_output = caption_model(**input_captions)
    new_outputs = torch.cat(
        [outputs.pooler_output, caption_output.pooler_output], dim=1
    )
    logits = classifier(new_outputs)
    for i in range(len(logits)):
        idx = logits[i].argmax(-1).item()
        answer = model.config.id2label[idx]
        answers.append(answer)

        results.append({"question_id": question_id[i], "answer": answer})

convert_data = vilt_to_vqa_annotation(new_dataset["test"])

convert_annotate, convert_questions = (
    convert_data["annotations"],
    convert_data["questions"],
)

with open("./annotate.json", "w") as annotate_file:
    annotate_file.write(json.dumps(convert_annotate))

with open("./questions.json", "w") as annotate_file:
    annotate_file.write(json.dumps(convert_questions))

with open("./results.json", "w") as res_file:
    res_file.write(json.dumps(results))

  0%|          | 0/625 [00:00<?, ?it/s]

In [ ]:
# for i, caption_addition in enumerate(caption_additions):
annFile = f"./annotate.json"
quesFile = f"./questions.json"
resFile = f"./results.json"

load_vqa = vqa.VQA(annFile, quesFile)
vqaRes = load_vqa.loadRes(resFile, quesFile)

myvqaEval = vqaEval.VQAEval(load_vqa, vqaRes, n=2)

myvqaEval.evaluate()

# print("Evaluation for caption addition: ", caption_addition)
print("Overall Accuracy is: %.02f\n" % (myvqaEval.accuracy["overall"]))
print("=====================")
print("Per Question Type Accuracy is the following:")
for quesType in myvqaEval.accuracy["perQuestionType"]:
    print(str(quesType) + " : " + str(myvqaEval.accuracy["perQuestionType"][quesType]))
print("=====================")
print("Per Answer Type Accuracy is the following:")
for ansType in myvqaEval.accuracy["perAnswerType"]:
    print(str(ansType) + " : " + str(myvqaEval.accuracy["perAnswerType"][ansType]))

loading VQA annotations and questions into memory...
0:00:07.206692
creating index...
index created!
Loading and preparing results...     
DONE
creating index...
index created!
computing accuracy
Finshed Percent: [####################] 98% Done computing accuracy
Overall Accuracy is: 78.29

Per Question Type Accuracy is the following:
which : 50.22
none of the above : 74.05
how : 38.57
what is : 64.29
how many : 69.09
what are : 76.67
is there : 96.22
what are the : 69.64
what type of : 67.32
is the : 91.63
are these : 95.57
do : 96.79
is this a : 94.78
what is the name : 25.0
are there any : 96.43
what : 57.42
are the : 92.17
is he : 96.56
what sport is : 92.76
is : 91.77
what color : 85.0
is it : 98.52
is the man : 88.6
what color is the : 89.51
what is the woman : 81.05
is there a : 95.95
what is the man : 82.57
what color is : 92.42
what is on the : 57.5
what time : 41.22
what kind of : 80.25
does the : 86.27
is this : 92.81
why is the : 46.67
has : 92.14
are there : 94.03
what is 